<a href="https://colab.research.google.com/github/jiminAn/2023_SUMMER_SAMSUNG_DL/blob/main/%EC%8B%A4%EC%8A%B5%EC%9E%90%EB%A3%8C/Day4/Day_4_3_RNN_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 4.3 RNN Basic
### 실습 목표
- RNN 모델의 입력 데이터 형태를 살펴보고 모델을 정의해보자

### Contents
1. RNN 모델의 입력 데이터 형태
2. RNN 모델 정의
  1. vanila RNN
  2. LSTM


## 0. Setting
- DL 관련 library 호출
- GPU 설정

In [ ]:
import torch
import numpy as np
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

cuda


## 1. RNN 모델의 입력 데이터 형태

### 예시: "hello"를 RNN 모델의 입력 데이터로 넣어보자.
![img.png](https://3863425935-files.gitbook.io/~/files/v0/b/gitbook-legacy-files/o/assets%2F-LIA3amopGH9NC6Rf0mA%2F-M4bJ-IWAKzglR0XHFwU%2F-M4bJ3KvyPXdSmZt6cU6%2Flanguage-model.png?generation=1586576265109629&alt=media)


In [ ]:
#"hello"를 one-hot encoding을 사용하여 벡터로 치환해봅시다
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [ ]:
input_data_np = np.array([[h, e, l, l, o]], dtype=np.float32)
input_data = torch.Tensor(input_data_np)
input_data

tensor([[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]]])

- 앞에서 선언한 벡터를 이용하여 추가로, "eolll", "lleel"를 벡터로 치환해봅시다

In [ ]:
# transform as torch tensor
input_data_np = np.array([[h, e, l, l, o],
                          [e, o, l, l, l],
                          [l, l, e, e, l]], dtype=np.float32)
input_data = torch.Tensor(input_data_np)
input_data

tensor([[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]],

        [[0., 1., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.]]])

In [ ]:
# (batch size, seq length(time step), input_size)
input_data.size()

torch.Size([3, 5, 4])

## 1. RNN 계열 모델 정의
1. Vanila RNN
2. LSTM




### 2.1 Vanila RNN

#### `torch.nn.RNN(input_size, hidden_size, batch_first)`
- [pytorch document](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)
- `input_size`: 입력해 주는 특성 값의 개수
- `hidden_size`: hidden state의 사이즈 지정. 보통 arbitrary 함.
- `batch_first`: 입력으로 받는 데이터의 shape중 첫 번째 차원을 batch로 간주할 것인지를 설정.
  - True: (batch, seq, feature)
  - False: (seq, batch, feature)

In [ ]:
# declare dimension
input_size = 4
hidden_size = 3

# declare RNN
rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)  # 처음에 batch_size or seq_length

In [ ]:
input_data

tensor([[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]],

        [[0., 1., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.]]])

In [ ]:
# check output
outputs, _status = rnn(input_data)

print("--------------모든 time steps의 hidden states-----------------")
print("outputs:\n", outputs)
print("outputs size:",outputs.size()) # (5번의 time step동안 3차원의 은닉 상태가 출력됨)*단어 3개
# shape : (3, 5, 3)
# (batch_size, seq_length, hidden_dim)
print("--------------마지막 시점의 hidden state-----------------")
print("hidden state:\n", _status) # 마지막 시점의 hidden state
print("hidden state size:",_status.size())
# shape: (1, 3, 3)
# (num_layers, batch_size, hidden_dim)

--------------모든 time steps의 hidden states-----------------
outputs:
 tensor([[[ 0.0714,  0.2574, -0.7411],
         [-0.2909,  0.6757, -0.5679],
         [-0.5539,  0.0745, -0.7523],
         [-0.5325,  0.2865, -0.5106],
         [-0.4065, -0.2141, -0.5597]],

        [[ 0.0460,  0.6588, -0.6216],
         [-0.3888, -0.2288, -0.7504],
         [-0.4218,  0.4226, -0.4555],
         [-0.4955,  0.1317, -0.6928],
         [-0.5153,  0.2671, -0.5565]],

        [[-0.1071,  0.3053, -0.7438],
         [-0.4803,  0.2826, -0.6808],
         [-0.4244,  0.5975, -0.4396],
         [-0.4117,  0.4884, -0.6059],
         [-0.5503,  0.1314, -0.6862]]], grad_fn=<TransposeBackward1>)
outputs size: torch.Size([3, 5, 3])
--------------마지막 시점의 hidden state-----------------
hidden state:
 tensor([[[-0.4065, -0.2141, -0.5597],
         [-0.5153,  0.2671, -0.5565],
         [-0.5503,  0.1314, -0.6862]]], grad_fn=<StackBackward0>)
hidden state size: torch.Size([1, 3, 3])


## 2.2 LSTM


#### `torch.nn.LSTM(input_size, hidden_size, batch_first)`
- [pytorch document](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)
- `input_size`: 입력해 주는 특성 값의 개수
- `hidden_size`: hidden state의 사이즈 지정. 보통 arbitrary 함.
- `batch_first`: 입력으로 받는 데이터의 shape중 첫 번째 차원을 batch로 간주할 것인지를 설정.
  - True: (batch, seq, feature)
  - False: (seq, batch, feature)

In [ ]:
lstm = torch.nn.LSTM(input_size, hidden_size, batch_first=True)
outputs, _status = lstm(input_data)
print(outputs)
print(outputs.size())
print()

print(_status[0])
print(_status[0].shape)

tensor([[[ 0.0817, -0.0826, -0.0224],
         [ 0.1010,  0.0253, -0.2162],
         [ 0.1177, -0.0613, -0.1277],
         [ 0.0883, -0.0982, -0.1151],
         [ 0.1481, -0.2102, -0.1580]],

        [[ 0.0812,  0.0761, -0.1982],
         [ 0.1724, -0.0886, -0.1710],
         [ 0.1256, -0.1003, -0.1442],
         [ 0.0917, -0.1174, -0.1270],
         [ 0.0732, -0.1245, -0.1212]],

        [[ 0.0426, -0.0602, -0.0579],
         [ 0.0561, -0.0932, -0.0878],
         [ 0.0856, -0.0051, -0.2485],
         [ 0.1095,  0.0300, -0.3012],
         [ 0.1155, -0.0629, -0.1533]]], grad_fn=<TransposeBackward0>)
torch.Size([3, 5, 3])

tensor([[[ 0.1481, -0.2102, -0.1580],
         [ 0.0732, -0.1245, -0.1212],
         [ 0.1155, -0.0629, -0.1533]]], grad_fn=<StackBackward0>)
torch.Size([1, 3, 3])
